In [5]:
# carregar base de dados
from test_helper import Test
import os.path
baseDir = os.path.join('Data')
inputPath = os.path.join('Aula04', 'BikeSharing.csv')
fileName = os.path.join(baseDir, inputPath)

numPartitions = 2
csvData = sc.textFile(fileName, numPartitions)
header = csvData.take(1)[0]
rawData = csvData.filter(lambda x: x!=header)

'''
https://www.kaggle.com/c/bike-sharing-demand
datetime - hourly date + timestamp  
season -  1 = spring, 2 = summer, 3 = fall, 4 = winter 
holiday - whether the day is considered a holiday
workingday - whether the day is neither a weekend nor holiday
weather - 1: Clear, Few clouds, Partly cloudy, Partly cloudy 
2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist 
3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds 
4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog 
temp - temperature in Celsius
atemp - "feels like" temperature in Celsius
humidity - relative humidity
windspeed - wind speed
casual - number of non-registered user rentals initiated
registered - number of registered user rentals initiated
count - number of total rentals
'''

'\nhttps://www.kaggle.com/c/bike-sharing-demand\ndatetime - hourly date + timestamp  \nseason -  1 = spring, 2 = summer, 3 = fall, 4 = winter \nholiday - whether the day is considered a holiday\nworkingday - whether the day is neither a weekend nor holiday\nweather - 1: Clear, Few clouds, Partly cloudy, Partly cloudy \n2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist \n3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds \n4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog \ntemp - temperature in Celsius\natemp - "feels like" temperature in Celsius\nhumidity - relative humidity\nwindspeed - wind speed\ncasual - number of non-registered user rentals initiated\nregistered - number of registered user rentals initiated\ncount - number of total rentals\n'

In [24]:

from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.linalg import DenseVector
from datetime import datetime as dt
import numpy as np
import math

def discrete(value, max, n):
    """Divide o intervalo de 0 até 'max' em 'n' intervalos de mesmo tamanho e retorna qual intervalo 'value' corresponde.
    
    Args:
        value: valor a ser discretizado.
        max: valor maximo que 'value' pode assumir.
        n: número de classes que serão levadas
    """
    
    if value==max: value-=1
    gap=max/n
    return int(math.floor(value/gap))

    

def parsePoint(point):
    """Converts a comma separated string into a list of (featureID, value) tuples.

    Note:
        featureIDs should start at 0 and increase to the number of features - 1.

    Args:
        point (str): A comma separated string where the first value is the label and the rest
            are features.

    Returns:
        list: A list of (featureID, value) tuples.
    """
    data = point.split(',')
    Date = dt.strptime(data[0], "%Y-%m-%d %H:%M:%S")
    DateList = [Date.year-2011, Date.month, Date.hour]   
    season = data[1]
    workingday=data[3]
    weather =data[4]
    sense=discrete(float(data[6]),46.,8)
    hum=discrete(float(data[7]),100.,8)
    wind=discrete(float(data[8]),57.,8)
    #features = DenseVector(DateList + season + realValues)
    features = DenseVector(DateList + [season]+ [workingday] + [sense] + [hum] + [wind] + [weather])# + realValues)
    
    label = discrete(int(data[-1]),997,10)
    
    return LabeledPoint(label, features)

weights = [.8, .1, .1]
seed = 42

rawTrainData, rawValData, rawTestData = rawData.randomSplit(weights, seed)

parsedTrainData = rawTrainData.map(parsePoint).cache()
parsedValData = rawValData.map(parsePoint).cache()
parsedTestData = rawTestData.map(parsePoint).cache()

print parsedTrainData.count()

8761


In [170]:
#Função própria do Spark
from pyspark.mllib.classification import NaiveBayes, NaiveBayesModel

# Train a naive Bayes model.
model = NaiveBayes.train(parsedTrainData, 1.0)

# Make prediction and test accuracy.
predictionAndLabel = parsedTestData.map(lambda p : (model.predict(p.features), p.label))
a=parsedTrainData.filter(lambda lp:lp.label==9)
print a.take(1)[0].label
model.predict(a.take(1)[0].features)
#accuracy = 1.0 * predictionAndLabel.filter(lambda (x, v): x == v).count() / parsedTestData.count()
#print accuracy

9.0


1.0

In [171]:
import numpy as np
def naiveBayesTrain(trainSet):
    """Recebe um set de treinos e calcula probabilidades e cria um modelo

    Args:
        trainSet (list)->(label, features)

    Returns:
        list: Um model, array[
                                Dict class : probabilidade(class),
                                RDD de tuplas (atributo, probabilidade(atributo))
                                RDD de tuplas ((class, atributo), probabilidade(atributo | class))
                                Int Quantidade de classes
                             ]
    """
    #Tiramos a quantidade de atributos de uma amostra
    amostra = trainSet.take(1)[0]
    #quantidade de atributos
    nAttri=len(amostra.features)
    
    RDDlen=trainSet.count()
    print RDDlen
    #Probabilidade de cada classe
    #Probabilidade de cada classe, sem logartimo, para uso na probabilidade condicional 
    #MODIFICADO: reaproveitamento de PClassesWithoutlog
    PClassesWithoutlog=(trainSet
                            .map(lambda lp:(lp.label,1))
                            .reduceByKey(lambda x,y:x+y)
                            .map(lambda (x,y):(x,y+1/float(RDDlen+nAttri))))
    PClassesRDD = (PClassesWithoutlog
                        .map(lambda (x,y):(x,np.log10(y))))
    #Podemos dar colect pra virar um dict, e facilitar o acesso, numero de classes nao sera grande suficiente para estourar
    PClassesWithoutlog = PClassesWithoutlog.collectAsMap()
    PClassesRDD = PClassesRDD.collectAsMap()
    

    
    
    #Probabilidade de cada atributo
    #lista de RDDs para cada atributo
    AttributesRDDs=[]
    #lista de RDDs para probabilidade de cada atributo dado classe
    AttributeClassesRDDs=[]
    
    #laço para percorrer os atributos
    #MODIFICADO: inclusão da normalização de termos nulo
    for i in range(nAttri):
        AttriRDD =(trainSet
                    .map(lambda lp:(lp.features[i],1))
                    .reduceByKey(lambda x,y:x+y)
                    .map(lambda (x,y):(x,np.log10((y+1)/float(RDDlen+nAttri))))
                  )
        AttributesRDDs.append(AttriRDD)
        AttriClassRDD=(trainSet
                        .map(lambda lp:((lp.label,lp.features[i]),1))
                        .reduceByKey(lambda x,y:x+y)
                        #.map(lambda ((c,a),q) : (x, np.log10((y+1)/float(RDDlen+nAttri))))
                        .map(lambda ((c,a),q) : ((c,a), np.log10(q+1/PClassesWithoutlog[c]+nAttri))) 
                      )
        AttributeClassesRDDs.append(AttriClassRDD)
        
    return [PClassesRDD, AttributesRDDs, AttributeClassesRDDs, nAttri]

def predict(model, objectToPredict):
    Pc = model[0]
    AttributesRDDs = model[1]
    AttributeClassesRDDs = model[2]
    nAttri = model[3]
    sumClass = -1
    bestClass = 0
    Pa = []
    Pac = []
    #Filtrar RDD de probabilidade para conter somente as que sao necessarias para o objeto a ser predito e entao criar um dict
    #MODIFICADO
    #gera uma lista com as prob. dos atributos
    for i in range(nAttri):
            Pa.extend((AttributesRDDs[i]
                            .filter(lambda (a,p): a == objectToPredict.features[i])
                            .map(lambda (a,p):p)).collect())
    print Pa
    sumPa=sum(Pa)
    
    for classe in Pc:
        Pac=[]
        for i in range(nAttri):
            #gera uma lista com probabilidades P(classe|objectToPredict.features[i])
            Pac.extend((AttributeClassesRDDs[i]
                        .filter(lambda ((c,a),p): a == objectToPredict.features[i] and c==classe)
                        .map(lambda ((c,a),p):p)).collect())
        # Soma segundo formula do slide 87
        soma=sum(Pac)+Pc[classe]-sumPa
        if(soma > sumClass):
            sumClass = soma
            bestClass = classe
            print bestClass
        print "Classe: "+str(classe)+" Soma:"+str(soma)
    return bestClass
print "built!"
        

built!


In [172]:
lst=naiveBayesTrain(parsedTrainData)
#gambiarra para tentar pegar amostra com label=9
a=parsedTrainData.filter(lambda lp:lp.label==9)
print a.take(1)[0].label
predict(lst,a.take(1)[0])
#TEM ALGUM ERRO QUE NÃO DEIXA A VALIDAÇÃO DE BAIXO SER CONCLUÍDA
#predictionAndLabel = parsedTestData.map(lambda p : (predict(lst,p.features), p.label))
#accuracy = 1.0 * predictionAndLabel.filter(lambda (x, v): x == v).count() / parsedTestData.count()
#print accuracy


8761
8.0
[-0.30043615626165271, -1.0761217790285418, -1.3771517746925228, -0.60834482648279919, -0.1655002737756755, -0.62036688167381704, -0.65296498200352249, -0.5664226363095286, -0.17912256187038564]
0.0
Classe: 0.0 Soma:34.4953145003
Classe: 1.0 Soma:32.6153049002
Classe: 2.0 Soma:31.5483837214
Classe: 3.0 Soma:29.4906796983
Classe: 4.0 Soma:27.5179856588
Classe: 5.0 Soma:26.2222023849
Classe: 6.0 Soma:23.7881761503
Classe: 7.0 Soma:22.1460859487
Classe: 8.0 Soma:21.0646867892
Classe: 9.0 Soma:15.3091376356


0.0